In [2]:
from bs4 import BeautifulSoup

In [5]:
from dotenv import load_dotenv
import os

load_dotenv("..")  # loads .env from current directory

api_id = int(os.getenv("API_ID"))
api_hash = os.getenv("API_HASH")